In [1]:
#Problem Statement : Derive Forward Kinematics for #R non planer Manipulator
#position of end Effector and  joints , Jw , Jv are express in frame 0

In [2]:
import numpy as np 
import math as m
import sympy  as sy

# Helper Function

In [3]:
class helper():
    def __init__ (self):
        self.id = 0
    @staticmethod
    def transform( lst ):
        a ,alpha, d , theta = lst[0],lst[1],lst[2],lst[3]
        alpha , theta = m.radians(alpha) , m.radians(theta)
        T = np.array ([ [ m.cos(theta)              , -1*m.sin(theta)           ,      0          ,    a              ],
                        [ m.sin(theta)*m.cos(alpha) , m.cos(theta)*m.cos(alpha) , -1*m.sin(alpha) , -1*m.sin(alpha)*d ],
                        [ m.sin(theta)*m.sin(alpha) , m.cos(theta)*m.sin(alpha) ,    m.cos(alpha) ,    m.cos(alpha)*d ],
                        [ 0                         , 0                         ,    0            ,  1                ]])
        return np.round(T, 3)
    
    @staticmethod 
    def sym_transform(lst):
        a ,alpha, d , theta = lst[0],lst[1],lst[2],lst[3]
        alpha , theta = m.radians(alpha) , m.radians(theta)
        T = sy.Matrix([ [ sy.cos(theta)              , -1*sy.sin(theta)           ,      0          ,    a              ],
                        [ sy.sin(theta)*sy.cos(alpha) , sy.cos(theta)*sy.cos(alpha) , -1*sy.sin(alpha) , -1*sy.sin(alpha)*d ],
                        [ sy.sin(theta)*sy.sin(alpha) , sy.cos(theta)*sy.sin(alpha) ,    sy.cos(alpha) ,    sy.cos(alpha)*d ],
                        [ 0                         , 0                         ,    0            ,  1                ]])
        return T
        
    @staticmethod 
    def sym_transform(a ,alpha, d , theta):
       
#         alpha , theta = m.radians(alpha) , m.radians(theta)
        T = sy.Matrix([ [ sy.cos(theta)              , -1*sy.sin(theta)           ,      0          ,    a              ],
                        [ sy.sin(theta)*sy.cos(alpha) , sy.cos(theta)*sy.cos(alpha) , -1*sy.sin(alpha) , -1*sy.sin(alpha)*d ],
                        [ sy.sin(theta)*sy.sin(alpha) , sy.cos(theta)*sy.sin(alpha) ,    sy.cos(alpha) ,    sy.cos(alpha)*d ],
                        [ 0                         , 0                         ,    0            ,  1                ]])
        return T
        

# Declering the symbols for joints and link lengths

In [4]:
theta1 = sy.Symbol('theta1')
theta2 = sy.Symbol('theta2')
theta3 = sy.Symbol('theta3')

In [5]:
Length_link_1 = sy.Symbol('L1')
Length_link_2 = sy.Symbol('L2')
Length_link_3 = sy.Symbol('L3')

#  Delcering the D_H parameter Matrix  (Useless)

In [6]:
D_H_parametrs = sy.Matrix([[0,0,0,theta1],[Length_link_1,-90,0,theta2],[Length_link_2,-90,0,theta3] , [Length_link_3,0,0,0]])

# Calculating the transformation Matrix 

In [7]:
# from fame 0 to 1
T01 = helper.sym_transform(0,0,0,theta1)
T01

Matrix([
[cos(theta1), -sin(theta1), 0, 0],
[sin(theta1),  cos(theta1), 0, 0],
[          0,            0, 1, 0],
[          0,            0, 0, 1]])

In [8]:
# from fame 1 to 2
T12 = helper.sym_transform(Length_link_1,-sy.pi/2,0,theta2).evalf()
T12

Matrix([
[ cos(theta2), -sin(theta2),   0,  L1],
[           0,            0, 1.0,   0],
[-sin(theta2), -cos(theta2),   0,   0],
[           0,            0,   0, 1.0]])

In [9]:
# from fame 0 to 2
T02 = T01 * T12
T02

Matrix([
[cos(theta1)*cos(theta2), -sin(theta2)*cos(theta1), -1.0*sin(theta1), L1*cos(theta1)],
[sin(theta1)*cos(theta2), -sin(theta1)*sin(theta2),  1.0*cos(theta1), L1*sin(theta1)],
[           -sin(theta2),             -cos(theta2),                0,              0],
[                      0,                        0,                0,            1.0]])

In [10]:
# from fame 2 to 3
T23 = helper.sym_transform(Length_link_2,-sy.pi/2,0,theta3).evalf()
T23

Matrix([
[ cos(theta3), -sin(theta3),   0,  L2],
[           0,            0, 1.0,   0],
[-sin(theta3), -cos(theta3),   0,   0],
[           0,            0,   0, 1.0]])

In [11]:
# from fame 3 to 4
T34 = helper.sym_transform(Length_link_3,0,0,0).evalf()
T34

Matrix([
[1.0,   0,   0,  L3],
[  0, 1.0,   0,   0],
[  0,   0, 1.0,   0],
[  0,   0,   0, 1.0]])

In [12]:
# from fame 0 to 3
T03 = T02*T23
T03

Matrix([
[1.0*sin(theta1)*sin(theta3) + cos(theta1)*cos(theta2)*cos(theta3),  1.0*sin(theta1)*cos(theta3) - sin(theta3)*cos(theta1)*cos(theta2), -1.0*sin(theta2)*cos(theta1), 1.0*L1*cos(theta1) + L2*cos(theta1)*cos(theta2)],
[sin(theta1)*cos(theta2)*cos(theta3) - 1.0*sin(theta3)*cos(theta1), -sin(theta1)*sin(theta3)*cos(theta2) - 1.0*cos(theta1)*cos(theta3), -1.0*sin(theta1)*sin(theta2), 1.0*L1*sin(theta1) + L2*sin(theta1)*cos(theta2)],
[                                         -sin(theta2)*cos(theta3),                                            sin(theta2)*sin(theta3),             -1.0*cos(theta2),                                 -L2*sin(theta2)],
[                                                                0,                                                                  0,                            0,                                             1.0]])

In [13]:
# from fame 0 to 4
T04 = T03*T34
T04

Matrix([
[1.0*sin(theta1)*sin(theta3) + 1.0*cos(theta1)*cos(theta2)*cos(theta3),  1.0*sin(theta1)*cos(theta3) - 1.0*sin(theta3)*cos(theta1)*cos(theta2), -1.0*sin(theta2)*cos(theta1), 1.0*L1*cos(theta1) + 1.0*L2*cos(theta1)*cos(theta2) + L3*(1.0*sin(theta1)*sin(theta3) + cos(theta1)*cos(theta2)*cos(theta3))],
[1.0*sin(theta1)*cos(theta2)*cos(theta3) - 1.0*sin(theta3)*cos(theta1), -1.0*sin(theta1)*sin(theta3)*cos(theta2) - 1.0*cos(theta1)*cos(theta3), -1.0*sin(theta1)*sin(theta2), 1.0*L1*sin(theta1) + 1.0*L2*sin(theta1)*cos(theta2) + L3*(sin(theta1)*cos(theta2)*cos(theta3) - 1.0*sin(theta3)*cos(theta1))],
[                                         -1.0*sin(theta2)*cos(theta3),                                            1.0*sin(theta2)*sin(theta3),             -1.0*cos(theta2),                                                                             -1.0*L2*sin(theta2) - L3*sin(theta2)*cos(theta3)],
[                                                                    0,                 

# Caalculating the positions for end effector  and joints

In [14]:
position_end_effector = T04[:3,3]
position_joint_3 = T03[:3,3]
position_joint_2 = T02[:3,3]
position_joint_1 = T01[:3,3]

# Initial Position Of End Effector

In [15]:
position_end_effector.subs({ Length_link_1:1,Length_link_2:1 ,Length_link_3:1,theta1:0,theta2:0,theta3:0 })

Matrix([
[3.0],
[  0],
[  0]])

# Position of End Effector In terms of Variables

In [16]:
T04[:3,3]

Matrix([
[1.0*L1*cos(theta1) + 1.0*L2*cos(theta1)*cos(theta2) + L3*(1.0*sin(theta1)*sin(theta3) + cos(theta1)*cos(theta2)*cos(theta3))],
[1.0*L1*sin(theta1) + 1.0*L2*sin(theta1)*cos(theta2) + L3*(sin(theta1)*cos(theta2)*cos(theta3) - 1.0*sin(theta3)*cos(theta1))],
[                                                                            -1.0*L2*sin(theta2) - L3*sin(theta2)*cos(theta3)]])

# Jacobian Matrix for Angular motion of End effector in frame 0

In [17]:
Jw = sy.Matrix([ list(T02[:3,2]) , list(T03[:3,2]) , list(T04[:3,2]) ])
Jw 

Matrix([
[            -1.0*sin(theta1),              1.0*cos(theta1),                0],
[-1.0*sin(theta2)*cos(theta1), -1.0*sin(theta1)*sin(theta2), -1.0*cos(theta2)],
[-1.0*sin(theta2)*cos(theta1), -1.0*sin(theta1)*sin(theta2), -1.0*cos(theta2)]])

# Jacobian Matrix for Linear motion of End effector in frame 0

In [18]:
Jv = sy.Matrix([ list(sy.diff(T04[:3,3],theta1)) , list(sy.diff(T04[:3,3],theta2)) , list(sy.diff(T04[:3,3],theta3)) ])
Jv

Matrix([
[-1.0*L1*sin(theta1) - 1.0*L2*sin(theta1)*cos(theta2) + L3*(-sin(theta1)*cos(theta2)*cos(theta3) + 1.0*sin(theta3)*cos(theta1)), 1.0*L1*cos(theta1) + 1.0*L2*cos(theta1)*cos(theta2) + L3*(1.0*sin(theta1)*sin(theta3) + cos(theta1)*cos(theta2)*cos(theta3)),                                                0],
[                                                      -1.0*L2*sin(theta2)*cos(theta1) - L3*sin(theta2)*cos(theta1)*cos(theta3),                                                     -1.0*L2*sin(theta1)*sin(theta2) - L3*sin(theta1)*sin(theta2)*cos(theta3), -1.0*L2*cos(theta2) - L3*cos(theta2)*cos(theta3)],
[                                                        L3*(1.0*sin(theta1)*cos(theta3) - sin(theta3)*cos(theta1)*cos(theta2)),                                                      L3*(-sin(theta1)*sin(theta3)*cos(theta2) - 1.0*cos(theta1)*cos(theta3)),                       L3*sin(theta2)*sin(theta3)]])